<a href = https://wikidocs.net/64515> PyTorch로 시작하는 딥 러닝 입문 _ Wikidocs</a>

<a href = https://velog.io/@dltjrdud37/%ED%95%9C%EA%B5%AD%EC%96%B4-%ED%86%A0%ED%81%B0%ED%99%94by-konlpy-%EB%8B%A8%EC%96%B4%EC%9E%84%EB%B2%A0%EB%94%A9Word2Vec> 한국어 토큰화 </a>

# 전처리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
# path = "/Users/sanghyun/Desktop/Coding/NLP/project/"

ly1 = pd.read_csv("/content/drive/MyDrive/nlp미니프로젝트/m0727.csv")
# ly2 = pd.read_csv(path + "melon_ballad(302).csv")
# ly3 = pd.read_csv(path + "melon_ballad(303).csv")
# ly4 = pd.read_csv(path + "melon_ballad(304).csv")
# ly5 = pd.read_csv(path + "melon_ballad(305).csv")

In [4]:
# 병합
raw_list = [ly1] #,ly2,ly3,ly4,ly5
df = pd.concat(raw_list)

# 번호 컬럼 drop
df.columns = ["ㅇㅇ","가수","제목","가사"]
df.drop(['ㅇㅇ'], axis=1, inplace=True)

# 중복 drop
df = df.drop_duplicates()

# 가수, 제목, 가사 없는거 drop
df.pop('가수')
df.pop('제목')
df.drop(df[df['가사']=='[가사 준비중]'].index, inplace=True)

# \n을 공백으로 변경
df['가사']= df['가사'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

df

,가사
0,잘 지냈지 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...
1,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래...
2,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는...
3,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던...
4,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 ...
...,...
2684,김현식의 노래에는 그대가 살고 있나 봐 내사랑 내 곁에 들으며 한잔 두잔 또 꺾어 ...
2685,어두운 조명 아래에 멈춰있는 우리 둘 이대로 멈췄으면 해 가만히 누워서 널 바라보는...
2686,널 처음 사진으로 본 그날 22년 2월 21일 그날 이후 지금 이 순간까지 나 하나...
2687,어쩜 우린 복잡한 인연에 서로 엉켜있는 사람인가봐 나는 매일 네게 갚지도 못할만큼 ...


- 여기까지가 merge 이후 단계

# Tokenization

In [5]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
lyrics = df.copy()
lyrics

,가사
0,잘 지냈지 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...
1,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래...
2,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는...
3,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던...
4,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 ...
...,...
2684,김현식의 노래에는 그대가 살고 있나 봐 내사랑 내 곁에 들으며 한잔 두잔 또 꺾어 ...
2685,어두운 조명 아래에 멈춰있는 우리 둘 이대로 멈췄으면 해 가만히 누워서 널 바라보는...
2686,널 처음 사진으로 본 그날 22년 2월 21일 그날 이후 지금 이 순간까지 나 하나...
2687,어쩜 우린 복잡한 인연에 서로 엉켜있는 사람인가봐 나는 매일 네게 갚지도 못할만큼 ...


In [7]:
# 영어 삭제
lyrics['가사'] = lyrics['가사'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
lyrics

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,가사
0,잘 지냈지 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...
1,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래...
2,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는...
3,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던...
4,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 ...
...,...
2684,김현식의 노래에는 그대가 살고 있나 봐 내사랑 내 곁에 들으며 한잔 두잔 또 꺾어 ...
2685,어두운 조명 아래에 멈춰있는 우리 둘 이대로 멈췄으면 해 가만히 누워서 널 바라보는...
2686,널 처음 사진으로 본 그날 년 월 일 그날 이후 지금 이 순간까지 나 하나만 기다려...
2687,어쩜 우린 복잡한 인연에 서로 엉켜있는 사람인가봐 나는 매일 네게 갚지도 못할만큼 ...


In [9]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 72.3 MB/s 


In [10]:
from konlpy.tag import Mecab, Hannanum, Kkma, Okt, Komoran

In [12]:
# 토큰화 함수 : 토큰 타입을 변경하고 싶으면 return 부분 수정
# Mecab..은 안쓸래... 

# hannanum = Hannanum()
# kkma = Kkma() #아연님
#komoran = Komoran() #태성님
okt = Okt() #성수님

def tokenizer(row):
    return okt.morphs(row)

모델 시간<br><img src="./img/modelTime.png" width="600" height="700">

In [13]:
lyrics['lyrics_token'] = lyrics['가사'].apply(tokenizer)
lyrics

,가사,lyrics_token
0,잘 지냈지 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...,"[잘, 지냈지, 조금, 은, 어색해, 요즘, 좋아, 보여, 인사, 나누며, 사실, ..."
1,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래...,"[우리, 다시, 만날, 래, 예쁘게, 빛나던, 모든, 추억, 들, 이, 너무, 그리..."
2,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는...,"[너, 와, 함께, 하고, 싶은, 일, 들, 을, 상상, 하는, 게, 요즘, 내, ..."
3,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던...,"[사랑, 이었다, 별거, 없던, 내, 하루, 에, 빛, 이, 돼, 준, 단, 한, ..."
4,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 ...,"[뭐, 하고, 있었니, 늦었지만, 잠시, 나올래, 너, 의, 집, 골목, 에, 있는..."
...,...,...
2684,김현식의 노래에는 그대가 살고 있나 봐 내사랑 내 곁에 들으며 한잔 두잔 또 꺾어 ...,"[김현식, 의, 노래, 에는, 그대, 가, 살, 고, 있나, 봐, 내사, 랑, 내,..."
2685,어두운 조명 아래에 멈춰있는 우리 둘 이대로 멈췄으면 해 가만히 누워서 널 바라보는...,"[어, 두운, 조명, 아래, 에, 멈춰있는, 우리, 둘, 이대로, 멈췄으면, 해, ..."
2686,널 처음 사진으로 본 그날 년 월 일 그날 이후 지금 이 순간까지 나 하나만 기다려...,"[널, 처음, 사진, 으로, 본, 그날, 년, 월, 일, 그날, 이후, 지금, 이,..."
2687,어쩜 우린 복잡한 인연에 서로 엉켜있는 사람인가봐 나는 매일 네게 갚지도 못할만큼 ...,"[어쩜, 우린, 복잡한, 인연, 에, 서로, 엉, 켜있는, 사람인, 가봐, 나, 는..."


In [21]:
lt = lyrics['lyrics_token'] 

In [22]:
print('최대 토큰 개수 : ',max(len(i) for i in lt))
print('평균 토큰 개수 : ',sum(map(len,lt))/len(lt))

최대 토큰 개수 :  524
평균 토큰 개수 :  163.13090368166604


# Embedding

- Embedding : text를 Vector로 변환시키는 방법 <br>
- <a href = "https://wikidocs.net/60854"> Word2Vec </a>

In [23]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=lyrics['lyrics_token'], size = 100, window = 5, min_count= 0, workers = 4, sg = 1)

In [24]:
model.wv.vectors.shape

(24470, 100)

In [38]:
# '나'와 유사한 키워드 추출

model_result = model.wv.most_similar('함께')
print(model_result)

[('둘이서', 0.736647367477417), ('함께라면', 0.7306373119354248), ('와', 0.7142200469970703), ('메리', 0.6945469975471497), ('나누고', 0.6795942783355713), ('함께해', 0.6776267290115356), ('듣던', 0.6744965314865112), ('함께한', 0.6722533702850342), ('걸었던', 0.6706854701042175), ('행복하고', 0.6703070998191833)]


In [39]:
df_11 = pd.DataFrame(model_result)
df_11

,0,1
0,둘이서,0.736647
1,함께라면,0.730637
2,와,0.714220
3,메리,0.694547
4,나누고,0.679594
5,함께해,0.677627
6,듣던,0.674497
7,함께한,0.672253
8,걸었던,0.670685
9,행복하고,0.670307
